<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_E1_Cassandra_Getting_Started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

#Install Cassandra

install https://phoenixnap.com/kb/install-cassandra-on-ubuntu

In [ ]:
!apt update > null
!apt install openjdk-8-jdk -y > null
#!java -version
!apt install apt-transport-https > null

In [ ]:
!sh -c 'echo "deb http://www.apache.org/dist/cassandra/debian 40x main" > /etc/apt/sources.list.d/cassandra.list'
!wget -q -O - https://www.apache.org/dist/cassandra/KEYS | sudo apt-key add -

OK


In [ ]:
!cat /etc/apt/sources.list.d/cassandra.list

deb http://www.apache.org/dist/cassandra/debian 40x main


In [ ]:
!apt update > null
!apt install cassandra > null

## Start the Service

In [ ]:
#!service cassandra stop
# the ulimit error does not matter
!service cassandra restart

 * Restarting Cassandra cassandra
/etc/init.d/cassandra: 72: ulimit: error setting limit (Operation not permitted)
   ...done.


In [ ]:
# keep checking service status until you see [+] next to cassandra
# then wait for 30 seconds MORE
!service  --status-all 

 [ + ]  cassandra
 [ - ]  dbus
 [ ? ]  hwclock.sh
 [ ? ]  kmod
 [ - ]  ntp
 [ - ]  procps
 [ - ]  rsync
 [ + ]  udev
 [ - ]  x11-common


In [ ]:
# this will give an error if the cassandra service has not started
# Wait for some time for the service to start
# If required reboot system or stop, restart cassandra service
!nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address    Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  127.0.0.1  69.09 KiB  16      100.0%            9a4ef910-0c78-4b6e-923c-94ed15f853eb  rack1



In [ ]:
# basic test
#!cqlsh

# Basic Commands 
how to execute commands through scripts https://stackoverflow.com/questions/25286253/executing-cql-through-shell-script <br>
some examples https://techfossguru.com/apache-cassandra-python-step-step-guide-ubuntu-example/

In [ ]:
!cqlsh -e "help" 


Documented shell commands:
CAPTURE  CLS          COPY  DESCRIBE  EXPAND  LOGIN   SERIAL  SOURCE   UNICODE
CLEAR    CONSISTENCY  DESC  EXIT      HELP    PAGING  SHOW    TRACING

CQL help topics:
AGGREGATES               CREATE_KEYSPACE           DROP_TRIGGER      TEXT     
ALTER_KEYSPACE           CREATE_MATERIALIZED_VIEW  DROP_TYPE         TIME     
ALTER_MATERIALIZED_VIEW  CREATE_ROLE               DROP_USER         TIMESTAMP
ALTER_TABLE              CREATE_TABLE              FUNCTIONS         TRUNCATE 
ALTER_TYPE               CREATE_TRIGGER            GRANT             TYPES    
ALTER_USER               CREATE_TYPE               INSERT            UPDATE   
APPLY                    CREATE_USER               INSERT_JSON       USE      
ASCII                    DATE                      INT               UUID     
BATCH                    DELETE                    JSON            
BEGIN                    DROP_AGGREGATE            KEYWORDS        
BLOB                     DROP_COLUMNF

In [ ]:
!cqlsh -e "CREATE KEYSPACE praxis WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 3 };"


Warnings :
Your replication factor 3 for keyspace praxis is higher than the number of nodes 1



In [ ]:
# since we are not specifying keyspace separately, we have to prefix table name with keyspace
!cqlsh -e "CREATE TABLE praxis.users (user_id int PRIMARY KEY, first_name text, last_name text );"

In [ ]:
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (1, 'Prithwis', 'Mukerjee');"
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (2, 'Sanniti', 'Chatterjee');"
!cqlsh -e "INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (3, 'Amrita', 'Ghosh');"

In [ ]:
!cqlsh -e "select * from praxis.users;"


 user_id | first_name | last_name
---------+------------+------------
       1 |   Prithwis |   Mukerjee
       2 |    Sanniti | Chatterjee
       3 |     Amrita |      Ghosh

(3 rows)


In [ ]:
#another way to specifiy keyspace
!cqlsh -e "use praxis;\
           select * from users;"


 user_id | first_name | last_name
---------+------------+------------
       1 |   Prithwis |   Mukerjee
       2 |    Sanniti | Chatterjee
       3 |     Amrita |      Ghosh

(3 rows)


In [ ]:
# easier way to specify keyspace, similar specifying database in mysql
!cqlsh -k praxis -e "select * from users;"


 user_id | first_name  | last_name
---------+-------------+------------
       1 | Dr Prithwis |   Mukerjee
       2 |     Sanniti | Chatterjee
       4 | Charanpreet |      Singh
       3 |      Amrita |      Ghosh

(4 rows)


# Python
https://docs.datastax.com/en/developer/python-driver/3.25/

In [ ]:
!pip install cassandra-driver

     |████████████████████████████████| 3.8 MB 4.1 MB/s 


In [ ]:
from cassandra.cluster import Cluster

In [ ]:
cluster = Cluster(['127.0.0.1'],port=9042)

In [ ]:
session = cluster.connect('praxis',wait_for_all_pools=True)

In [ ]:
session.execute('USE praxis')

In [ ]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Prithwis Mukerjee
2 Sanniti Chatterjee
3 Amrita Ghosh


In [ ]:
session.execute("INSERT INTO praxis.users (user_id, first_name, last_name) VALUES (4, 'Charanpreet', 'Singh');")

In [ ]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Prithwis Mukerjee
2 Sanniti Chatterjee
4 Charanpreet Singh
3 Amrita Ghosh


In [ ]:
session.execute("update praxis.users set first_name = 'Dr Prithwis' where user_id = 1;")

In [ ]:
rows = session.execute('SELECT * FROM users')
for row in rows:
        print(row.user_id,row.first_name,row.last_name)

1 Dr Prithwis Mukerjee
2 Sanniti Chatterjee
4 Charanpreet Singh
3 Amrita Ghosh


#Tutorial from FreeCodeCamp <br>
https://www.freecodecamp.org/news/the-apache-cassandra-beginner-tutorial/ <br>
alternatively from this [saved file](https://github.com/Praxis-QR/BDSN/blob/main/Documents/The%20Apache%20Cassandra%20Beginner%20Tutorial.pdf)

In [ ]:
#!cqlsh -e "CREATE KEYSPACE learn_cassandra WITH REPLICATION = { 'class' : 'NetworkTopologyStrategy','datacenter1' : 3  };"

In [ ]:
!cqlsh -e "CREATE TABLE praxis.users_by_country \
(country text,\
 user_email text,\
 first_name text,\
 last_name text,\
 age smallint,\
 PRIMARY KEY ((country), user_email));"

In [ ]:
!cqlsh -e "\
INSERT INTO praxis.users_by_country (country,user_email,first_name,last_name,age) \
  VALUES('US', 'john@email.com', 'John','Wick',55); \
\
INSERT INTO praxis.users_by_country (country,user_email,first_name,last_name,age)\
  VALUES('UK', 'peter@email.com', 'Peter','Clark',65);\
\
INSERT INTO praxis.users_by_country (country,user_email,first_name,last_name,age)\
  VALUES('UK', 'bob@email.com', 'Bob','Sandler',23);\
\
INSERT INTO praxis.users_by_country (country,user_email,first_name,last_name,age)\
  VALUES('UK', 'alice@email.com', 'Alice','Brown',26);\
"

In [ ]:
!cqlsh -e "SELECT * FROM praxis.users_by_country WHERE country='UK';"


 country | user_email      | age | first_name | last_name
---------+-----------------+-----+------------+-----------
      UK | alice@email.com |  26 |      Alice |     Brown
      UK |   bob@email.com |  23 |        Bob |   Sandler
      UK | peter@email.com |  65 |      Peter |     Clark

(3 rows)


In [ ]:
!cqlsh -e "drop table praxis.users_by_email;"
!cqlsh -e "CREATE TABLE praxis.users_by_email (\
  user_email text,\
  country text,\
  first_name text,\
  last_name text,\
  age smallint,\
  PRIMARY KEY (user_email));"
  

In [ ]:
!cqlsh -e "INSERT INTO praxis.users_by_email (user_email, country,first_name,last_name,age)  VALUES('john@email.com', 'US', 'John','Wick',55);"
!cqlsh -e "INSERT INTO praxis.users_by_email (user_email,country,first_name,last_name,age)  VALUES('peter@email.com', 'UK', 'Peter','Clark',65);"
!cqlsh -e "INSERT INTO praxis.users_by_email (user_email,country,first_name,last_name,age)  VALUES('bob@email.com', 'UK', 'Bob','Sandler',23);"
!cqlsh -e "INSERT INTO praxis.users_by_email (user_email,country,first_name,last_name,age)  VALUES('alice@email.com', 'UK', 'Alice','Brown',26);"

In [ ]:
!cqlsh -e "SELECT * FROM praxis.users_by_email WHERE user_email='alice@email.com';"


 user_email      | age | country | first_name | last_name
-----------------+-----+---------+------------+-----------
 alice@email.com |  26 |      UK |      Alice |     Brown

(1 rows)


In [ ]:
!cqlsh -e "SELECT * FROM praxis.users_by_email WHERE age=26 ALLOW FILTERING;"
!cqlsh -e "SELECT * FROM praxis.users_by_email WHERE age=26;"


 user_email      | age | country | first_name | last_name
-----------------+-----+---------+------------+-----------
 alice@email.com |  26 |      UK |      Alice |     Brown

(1 rows)
<stdin>:1:InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"


In [ ]:
!cqlsh -e "CREATE TABLE praxis.todo_by_user_email (user_email text,name text,creation_date timestamp,PRIMARY KEY ((user_email), creation_date)) WITH CLUSTERING ORDER BY (creation_date DESC) AND compaction = { 'class' :  'LeveledCompactionStrategy'};"

In [ ]:
!cqlsh -e "\
CREATE TABLE praxis.todos_shared_by_target_user_email ( \
    target_user_email text, \
    source_user_email text, \
    creation_date timestamp, \
    name text, \
    PRIMARY KEY ((target_user_email), creation_date) \
) WITH CLUSTERING ORDER BY (creation_date DESC) \
AND compaction = { 'class' :  'LeveledCompactionStrategy'  };"

In [ ]:
!cqlsh -e "\
CREATE TABLE praxis.todos_shared_by_source_user_email ( \
    target_user_email text, \
    source_user_email text, \
    creation_date timestamp, \
    name text, \
    PRIMARY KEY ((source_user_email), creation_date) \
) WITH CLUSTERING ORDER BY (creation_date DESC) \
AND compaction = { 'class' :  'LeveledCompactionStrategy'  };"

In [ ]:
!cqlsh -e "\
BEGIN BATCH \
  INSERT INTO praxis.todo_by_user_email (user_email,creation_date,name) VALUES('alice@email.com', toTimestamp(now()), 'My first todo entry') \
  INSERT INTO praxis.todos_shared_by_target_user_email (target_user_email, source_user_email,creation_date,name) VALUES('bob@email.com', 'alice@email.com',toTimestamp(now()), 'My first todo entry') \
  INSERT INTO praxis.todos_shared_by_source_user_email (target_user_email, source_user_email,creation_date,name) VALUES('alice@email.com', 'bob@email.com', toTimestamp(now()), 'My first todo entry') \
APPLY BATCH;"

In [ ]:
!cqlsh -e "SELECT * FROM praxis.todos_shared_by_target_user_email WHERE target_user_email='bob@email.com';"


 target_user_email | creation_date                   | name                | source_user_email
-------------------+---------------------------------+---------------------+-------------------
     bob@email.com | 2021-11-27 07:10:16.640000+0000 | My first todo entry |   alice@email.com

(1 rows)


In [ ]:
!cqlsh -e "SELECT * FROM praxis.todo_by_user_email WHERE user_email='alice@email.com';"


 user_email      | creation_date                   | name
-----------------+---------------------------------+---------------------
 alice@email.com | 2021-11-27 07:10:16.639000+0000 | My first todo entry

(1 rows)


#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)